In [ ]:
# importamos las librerias necesarias
import pandas as pd
import numpy as np
import os
import json
import pickle
import gzip
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    balanced_accuracy_score,
    recall_score,
    f1_score,
)
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.svm import SVC


In [ ]:
# Funciones auxiliares

def crear_directorios_base():
    """Asegura que los directorios de salida existan."""
    os.makedirs("../files/models", exist_ok=True)
    os.makedirs("../files/output", exist_ok=True)

def cargar_datos_fuente():
    """Carga los DataFrames de entrenamiento y prueba."""
    train_df = pd.read_csv("../files/input/train_data.csv.zip")
    test_df = pd.read_csv("../files/input/test_data.csv.zip")
    return train_df, test_df

def guardar_modelo_comprimido(estimator, path="../files/models/model.pkl.gz"):
    """Guarda el estimador comprimido con gzip."""
    crear_directorios_base()
    with gzip.open(path, "wb") as f:
        pickle.dump(estimator, f)

def recuperar_modelo(path="../files/models/model.pkl.gz"):
    """Carga el estimador guardado si existe, sino devuelve None."""
    if not os.path.exists(path):
        return None
    with gzip.open(path, "rb") as f:
        return pickle.load(f)

In [ ]:
# Funciones para limpiar y cargar los datos

def limpiar_dataframe(df):
    """Limpia y transforma un DataFrame (manejo de columnas y valores atípicos)."""
    df = df.copy()

    # renombrar target
    df.rename(columns={"default payment next month": "default"}, inplace=True)

    # eliminar ID
    if "ID" in df.columns:
        df.drop(columns=["ID"], inplace=True)

    # eliminar EDUCATION = 0 y agrupar valores > 4 en 4
    df = df[df["EDUCATION"] != 0]
    df["EDUCATION"] = df["EDUCATION"].apply(lambda x: 4 if x > 4 else x)

    # eliminar MARRIAGE = 0
    df = df[df["MARRIAGE"] != 0]

    # eliminar NaNs
    df = df.dropna()

    return df

In [ ]:
# division
def obtener_splits_entrenamiento_prueba(train_df, test_df):
    """
    Aplica limpiar_dataframe y retorna x_train, y_train, x_test, y_test
    """
    train_clean = limpiar_dataframe(train_df)
    test_clean = limpiar_dataframe(test_df)

    x_train = train_clean.drop(columns=["default"])
    y_train = train_clean["default"]

    x_test = test_clean.drop(columns=["default"])
    y_test = test_clean["default"]

    return x_train, y_train, x_test, y_test

In [ ]:
# Funciones parael modelo

def construir_pipeline_completo(feature_columns):
    """
    Construye el pipeline de preprocesamiento y el modelo MLP.
    Orden del Código Fuente: (OHE + StandardScaler) -> SelectKBest -> PCA -> MLP
    """
    categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]
    # Las demás columnas se consideran numéricas
    numeric_features = [c for c in feature_columns if c not in categorical_features]

    # ColumnTransformer: OHE para categóricas, StandardScaler para numéricas (como en el Código Fuente)
    preprocessor = ColumnTransformer(
        [
            ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features),
            ("num", StandardScaler(), numeric_features),
        ],
        remainder='passthrough'
    )

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            # Orden del Código Fuente: SelectKBest va antes de PCA
            ("feature_selection", SelectKBest(score_func=f_classif)),
            ("pca", PCA()),
            # Hiperparámetros de MLP del Código Fuente
            ("mlp", MLPClassifier(
                max_iter=15000,
                random_state=21 # Random state del Código Fuente
                )
            )
        ]
    )

    return pipeline

def configurar_busqueda_grid(estimator, param_grid, cv=10):
    """Crea y configura el objeto GridSearchCV."""
    # cv=10 como en el Código Fuente (entero, que por defecto es KFold)
    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring="balanced_accuracy",
        n_jobs=-1,
        verbose=1,
        refit=True
    )
    return grid_search


In [ ]:
# Funciones de Entrenamiento y Validación

def entrenar_y_comparar_modelos(grid_search):
    """
    Entrena grid_search con los datos, compara con modelo guardado (si existe)
    usando balanced_accuracy en el conjunto de test; mantiene el mejor.
    """
    train_df, test_df = cargar_datos_fuente()
    x_train, y_train, x_test, y_test = obtener_splits_entrenamiento_prueba(train_df, test_df)

    # entrenar
    grid_search.fit(x_train, y_train)

    # cargar modelo guardado (si existe) y comparar en balanced_accuracy sobre test
    saved = recuperar_modelo()
    current_score = balanced_accuracy_score(y_test, grid_search.predict(x_test))

    saved_score = -1.0
    if saved is not None:
        try:
            saved_score = balanced_accuracy_score(y_test, saved.predict(x_test))
        except Exception:
            saved_score = -1.0 # si el objeto guardado no tiene predict (o fue guardado mal)


    if current_score >= saved_score:
        # guardar el grid_search (fitted) para revisar cv_results etc.
        guardar_modelo_comprimido(grid_search)
    else:
        # mantener el guardado (no sobreescribimos)
        pass

def ejecutar_entrenamiento_mlp():
    """Define la cuadrícula de parámetros para MLP y ejecuta el entrenamiento."""
    train_df, test_df = cargar_datos_fuente()
    x_train, y_train, x_test, y_test = obtener_splits_entrenamiento_prueba(train_df, test_df)

    pipeline = construir_pipeline_completo(feature_columns=x_train.columns.tolist())

    # Parámetros del Código Fuente
    param_grid = {
    'feature_selection__k': [20], # numero de columnas a seleccionar (Código Fuente)
    'pca__n_components': [None],  # usar todas las componentes (Código Fuente)
    'mlp__hidden_layer_sizes': [(50, 30, 40, 60)], # capas y neuronas ocultas (Código Fuente)
    'mlp__alpha': [0.26], # regularización L2 (Código Fuente)
    'mlp__learning_rate_init': [0.001], # tasa de aprendizaje inicial (Código Fuente)
    }

    # cv=10 como en el Código Fuente.
    gs = configurar_busqueda_grid(estimator=pipeline, param_grid=param_grid, cv=10)
    entrenar_y_comparar_modelos(gs)

In [ ]:
# validacion y métricas

def validar_y_generar_metricas():
    """Carga el modelo final, calcula y guarda todas las métricas en un archivo JSONL."""
    crear_directorios_base()
    train_df, test_df = cargar_datos_fuente()
    x_train, y_train, x_test, y_test = obtener_splits_entrenamiento_prueba(train_df, test_df)

    # cargar modelo (gzip)
    estimator = recuperar_modelo()
    if estimator is None:
        raise FileNotFoundError("No se encontró modelo en files/models/model.pkl.gz")

    # predicciones
    y_train_pred = estimator.predict(x_train)
    y_test_pred = estimator.predict(x_test)

    metrics = []

    # Métricas de entrenamiento
    train_metrics = {
        "type": "metrics",
        "dataset": "train",
        "precision": precision_score(y_train, y_train_pred, zero_division=0),
        "balanced_accuracy": balanced_accuracy_score(y_train, y_train_pred),
        "recall": recall_score(y_train, y_train_pred, zero_division=0),
        "f1_score": f1_score(y_train, y_train_pred, zero_division=0),
    }
    metrics.append(train_metrics)

    # Métricas de prueba
    test_metrics = {
        "type": "metrics",
        "dataset": "test",
        "precision": precision_score(y_test, y_test_pred, zero_division=0),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
        "recall": recall_score(y_test, y_test_pred, zero_division=0),
        "f1_score": f1_score(y_test, y_test_pred, zero_division=0),
    }
    metrics.append(test_metrics)

    # matriz de confusión train
    cm_train = confusion_matrix(y_train, y_train_pred)
    cm_train_dict = {
        "type": "cm_matrix",
        "dataset": "train",
        "true_0": {"predicted_0": int(cm_train[0, 0]), "predicted_1": int(cm_train[0, 1])},
        "true_1": {"predicted_0": int(cm_train[1, 0]), "predicted_1": int(cm_train[1, 1])},
    }
    metrics.append(cm_train_dict)

    # matriz de confusión test
    cm_test = confusion_matrix(y_test, y_test_pred)
    cm_test_dict = {
        "type": "cm_matrix",
        "dataset": "test",
        "true_0": {"predicted_0": int(cm_test[0, 0]), "predicted_1": int(cm_test[0, 1])},
        "true_1": {"predicted_0": int(cm_test[1, 0]), "predicted_1": int(cm_test[1, 1])},
    }
    metrics.append(cm_test_dict)

    # guardar JSONL
    out_path = "../files/output/metrics.json"
    with open(out_path, "w") as f:
        for m in metrics:
            f.write(json.dumps(m) + "\n")

    print(f"Métricas guardadas en {out_path}")


if __name__ == "__main__":
    # si se ejecuta el script, entrena y luego comprueba
    crear_directorios_base()
    ejecutar_entrenamiento_mlp()
    validar_y_generar_metricas()